### Setup

<b>Objective</b><br>
To showcase the minimum number of steps<br>
required to do tertiary analysis of DNA + Protein<br>
and some of the different ways to look at the data<br>

<b>Major questions answered:</b>
1. Do we see DNA clones?<br>
2. Do we see protein cell types<br>
3. Is the differential expression significant?<br>
4. Do the clones correlate with the cell types?<br>

<b>Things not shown:</b>
1. All available methods eg. Filtering of nearby variants, variant annotation, plots

2. Discussing all methods and their options - Documented [here](https://mbkkjsp.github.io/mexdocs/index.html)<br>

3. Systemic variations seen in protein data - Next session<br>

In [1]:
#import mosaic libraries

import missionbio.mosaic.io as mio

In [2]:
help(mio.load)

Help on function load in module missionbio.mosaic.io:

load(filepath: Any, apply_filter: bool = True, whitelist: Union[Sequence, NoneType] = None, raw: bool = False, single: bool = False) -> Union[missionbio.mosaic.sample.Sample, missionbio.mosaic.samplegroup.SampleGroup]
    Loading the .h5 file with one or more assays.
    
    This is the preferred way of loading .h5 files.
    
    It directly returns a `Sample` object, which
    contains all the assays. Those assays that were
    not present are stored as `None`.
    
    Parameters
    ----------
    filepath:
        The path to the .h5 multi-omics file.
    apply_filter:
        Whether to load only the filtered dna variants.
    whitelist:
        The specific dna variants to load.
    raw:
        Whether the raw counts are to be loaded.
    single:
        Whether to load as a single sample despite being
        a multi sample h5 file (Not a recommended approach).
        Only use this for debugging issues.
    
    Returns


In [5]:
#h5path = r'C:\Users\marribas\Documents\Loom h5 files\UPenn - Saar Gill\No4771_DNA_Protein.dna+protein.h5'
#if working with Windows you may need to add an r before the path: h5path = r'/path/to.h5/file/test.h5'
sample = mio.load("/media/daisuke-ido/Seagate Expansion Drive/Gill_Tapestri/data/No4771_DNA_Protein.dna+protein.h5", raw=False, apply_filter=True) 
#keep raw always at False; if raw = True ALL barcodes will be loaded (not just cell assoc barcodes)
#keep apply_filter at True unless you can't detect an expected (target) variant

OSError: Unable to open file (unable to open file: name = '/media/daisuke-ido/Seagate Expansion Drive/Gill_Tapestri/data/No4771_DNA_Protein.dna+protein.h5', errno = 30, error message = 'Read-only file system', flags = 1, o_flags = 2)

In [ ]:
#import graph_objects from the plotly package to be able to save and display figures when saving the notebook in HTML.
import plotly.graph_objects as go

All the interactive plotting functions return a plotly figure. In case the layout or the color
scheme is not suitable for your data type, they can be changed before creating the final figure.

The color for the plots are store either in the individual traces or the layout attributes of the plotly figure.

Mosaic also contains a list of colors that can be used to customize the plots.

In [ ]:
# Import the colors
from missionbio.mosaic.constants import COLORS

# additional color palettes: https://seaborn.pydata.org/tutorial/color_palettes.html

# Plot the first few colors
import seaborn as sns
sns.palplot(COLORS[:40])


In [ ]:
help(sns.color_palette)

In [ ]:
# Alternatively plot another palette

sns.palplot(sns.color_palette("magma", n_colors=20))

In [ ]:
#hex codes
sns.color_palette("viridis", n_colors=20).as_hex()

### Data structure

    Dna, Cnv, and Protein are sub classes of the Assay class
    The information is stored in four ways, and the user
    can change each of those

    1. metadata (add_metadata / del_metadata):
        dictionary containing metrics of the assay

    2. row_attrs (add_row_attr / del_row_attr):
        dictionary which contains 'barcode' as one of
        the keys. All the values must be of the same
        length i.e. match the number of barcodes
        This is the attribute where 'label', 'pca',
        and 'umap' values are added

    3. col_attrs (add_col_attr / del_col_attr):
        dictionary which contains 'ids' as one of
        the keys. All the values must be of the same
        length i.e. match the number ids
        'ids' contains variants for DNA assays
        and anitobides for Protein assays

    4. layers (add_layer / del_layer):
        dictionary containing 'read_counts' as one of 
        the metrics. All the values have the shape
        (num barcodes) x (num ids). This is the attribute
        where 'normalized_counts' will be added

    Sample holds the Dna and Protein information
    



In [ ]:
sample.protein

In [ ]:
sample.dna

In [ ]:
sample.dna.metadata

In [ ]:
#data frame rownames, eg protein
sample.protein.row_attrs

In [ ]:
#data frame column names; eg protein
sample.protein.ids()

In [ ]:
#object layers, eg dna; comparable to Tapestri Insights Advanced Filters 
sample.dna.layers

### DNA Analysis

<b>Topics covered</b><br>
1. Whitelist of variants
2. Manually selecting variants

#### Basic filtering

    Many filtering options are available
    use the documentation shared earlier,
    or the help() function to get the same
    information here

In [ ]:
#note any function's parameters and their default values can be looked up via the help function. 
#Here the function is filter_variants
#For additional information re: Advanded Filters visit 
#https://support.missionbio.com/hc/en-us/articles/360047303654-How-do-the-Advanced-Filters-work-
help(sample.dna.filter_variants)

In [ ]:
# Filter variants
# Comparable to TI v2.2 Advanced Filters
#Difference: min_std (based on VAF), leave max_vaf at 100 at all times

#define dna_vars variable
dna_vars = sample.dna.filter_variants()
#output
dna_vars

In [ ]:
# Check the number of filtered variants
len(dna_vars)

In [ ]:
# adjust filters if needed
#overwriting dna_vars with adjusted filters

#note if you want to work exclusively with whitelisted variants, set one filter to 100, e.g., min_std=100 
#--> this will create an empty array
dna_vars = sample.dna.filter_variants(min_dp=10, min_gq=30, vaf_ref=5, vaf_hom=95, min_prct_cells=50, min_mut_prct_cells=1) 
#output
dna_vars

In [ ]:
len(dna_vars)

#### Whitelist

    Simply appnding the whitelist to the list of filtered
    variants is sufficient to then select the variants
    using the slice notation
    
    i.e. sample.dna[{list of barcodes}, {list of ids}]

In [ ]:
#based on TI; may be copy/pasted from TI; ensure correct nomenclature
#whitelist = ["chr7:6398156,3:A/G,""chr7:147,674978:A/G","chr7:128371,206:G/A"]

#ADD VARIANTS OF INTEREST HERE

whitelist = []

In [ ]:
#update final variant list
final_vars = list(dna_vars) + whitelist

In [ ]:
len(final_vars)

In [ ]:
#dimensionality of sample.dna dataframe
# first number = number of cells (rows), second number = number of pre-filtered variants (columns) as in vcf.gz file
sample.dna.shape

In [ ]:
# Subsetting sample.dna based on reduced variant (column) list. Selecting all cells and final variants
sample.dna = sample.dna[sample.dna.barcodes(), final_vars]

In [ ]:
# Check the shape i.e. (Number of barcodes, number of ids)
# of the final filtered dna object
sample.dna.shape

#### Manual variant selection

    Heatmaps are interactive. Clicking on it selects
    the corresponding id whose value is stored in the
    `selected_ids` attribute of the object
    
    eg. sample.dna.selected_ids

In [ ]:
help(sample.dna.get_annotations)

In [ ]:
#Add annotation to the SNV using varsome (adds gene name, AA changes to variants)
#import missionbio.mosaic.utils as mutils
ann_ids = sample.dna.get_annotations()  # Run this on a filtered DNA sample - too many variants are not handled correctly by the method
sample.dna.add_col_attr('id', ann_ids)

In [ ]:
#annotations now added to the variants
sample.dna.ids()

In [ ]:
help(sample.dna.stripplot)

In [ ]:
## First diagnostic plot to evaluate variant quality. attribute or colorby arguments may be changed
#adding go.Figure(fig) command so that the figure is saved when exporting the notebook to HTML
#adding write_image function saves the picture in high-res (commented out below)
fig = sample.dna.stripplot(attribute='AF_MISSING', colorby='NGT')
go.Figure(fig)
#fig.write_image("stripplot_example.png") # change resolution or file type via help(fig.write_image)

In [ ]:
sample.dna.filter_variants()

In [ ]:
#2nd diagnostic plot: use this heatmap to (de)-select variants with little variance 
#(e.g., germline mut w/ identical genotype across all cells) to be discarded later
fig = sample.dna.heatmap(attribute="")
go.Figure(fig)

In [ ]:
#redraw w/ GQ values
#this heatmap may also be used to select variants that may display low quality in majority of cells.
#Don't use "fig =" as the heatmap won't become responsive for variant selection.
#variants may be selected by clicking on the heatmap (variant name changes color from black to red)
sample.dna.heatmap(attribute='GQ')


In [ ]:
#array that lists all variants that were selected in the heatmap
sample.dna.selected_ids

In [ ]:
#subsetting the sample.dna variable by removing all selected variants
#sample.dna = sample.dna.drop(sample.dna.selected_ids)

In [ ]:
#confirm new # of columns (variables)
sample.dna.shape

In [ ]:
#redraw heatmap with variants removed
sample.dna.heatmap(attribute='NGT')

#### Clustering

    DNA has a custom clustering method called `find_clones`
    
    It projects the data on a UMAP and then performs
    dbscan to identify unique clusters, which are then
    merged in case they were formed due to missing
    information

In [ ]:
help(sample.dna.find_clones)

In [ ]:
#clustering method #1 - useful for high-feature/dimensional space (e.g., 10+ variants used for downstream analysis)

#standard method uses 2 sequential dimension reduction techniques: PCA on filtered data frame (all cells + filtered variant)
#followed by UMAP
#on transformed PC dataframe --> only suggested for high-dimensional data
#if only 1-5 variants are in your sample.dna (typical for most primary samples)
#use a different method to cluster (see further below)
sample.dna.find_clones()

In [ ]:
#verify that another row attribute was added = umap, which corresponds to a cluster number for each cell (barcode)
sample.dna.row_attrs

In [ ]:
fig = sample.dna.scatterplot(attribute='umap', colorby='label')
go.Figure(fig)

In [ ]:
#display column names so you can color code umap not on labels, but on variant specific data (e.g., AF) 
sample.dna.ids()

In [ ]:
#replot umap with variant information
fig = sample.dna.scatterplot(attribute='umap', colorby='NGT',features=sample.dna.ids())
go.Figure(fig)

In [ ]:
#remove umap attribute in order to follow up with an alternative clustering method
sample.dna.del_row_attr('umap')

In [ ]:
#clustering method #2

#cluster w/ alternative method compared to find_clones()
#here you can specify and customize arguments in each step that is integrated in the find_clones() f(x) 
#1 run PCA on data frame using AF_MISSING for instance
#(note, AF_MISSING includes -50 values for all 0 values for which the data was filtered and is missing)
sample.dna.run_pca(components=8, attribute='AF_MISSING',show_plot=True) # component # should equal # of PF variants
# Assess 'elbow' plot and determine number of PCs where the distribution starts to plateau. Note that PC1 = 0 in the plot

In [ ]:
#rerun PCA w/ optimal PC number based on elbow plot analysis
sample.dna.run_pca(components=4, attribute='AF_MISSING')

In [ ]:
#run UMAP on top of the newly created PC dataframe. 
#See https://jlmelville.github.io/uwot/abparams.html for appropriate spread/min_dist values
sample.dna.run_umap(attribute='pca', min_dist=0.2, spread=1.5, random_state=40)

In [ ]:
#visualize data and cluster it using different methods
sample.dna.cluster(attribute='umap', method='dbscan')#, eps=0.8)

In [ ]:
help(sample.dna.cluster)

In [ ]:
#replot UMAP projection w/ alternative clustering results
fig = sample.dna.scatterplot(attribute='umap', colorby='label')
go.Figure(fig)

In [ ]:
help(sample.dna.count) 

In [ ]:
#clustering method #3

#a third alternative method to cluster your data. recommended for 1-5 variants
#cluster with TI v2.2 count-based method 
#note, the umap attr. does not have to be removed a prior as the count based clustering does not create a umap attr.

#The created table includes a 'score' column that intends to help with the identification of allele dropout (ADO) clones
# Scores greater than 0.8 are typically considered artifacts (ADO) and may be labeled as such to be discarded.
# Clones are ordered based on their size (1=largest clone, 2=second largest clone, etc)
sample.dna.count(features=sample.dna.ids(),group_missing=True,min_clone_size=0.9,layer="NGT_FILTERED")

In [ ]:
#plot heatmap using NGT
fig = sample.dna.heatmap(attribute='NGT_FILTERED')
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
#plot heatmap using NGT w/ barcodes ordered based on sample
fig = sample.dna.heatmap(attribute='NGT',splitby="sample_name")
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
help(sns.color_palette)

In [ ]:
sns.palplot(sns.color_palette("viridis", n_colors=20))

In [ ]:
vir20 = sns.color_palette("viridis", n_colors=20).as_hex()
vir20

In [ ]:
vir20[0]

In [ ]:
# In case of the DNA heatmap and scatterplot the colors are
# stored in the layout.coloraxis.colorscale attribute.

# This value must be updated to customize the plot.

fig.layout.coloraxis.colorscale

In [ ]:
# Assuming these are new desired colors
# NGT=0 (WT) - blue
# NGT=1 (HET) - orange
# NGT=2 (HOM) - red
# NGT=3 (missing) - black

# Additional information re: color palettes here: https://seaborn.pydata.org/tutorial/color_palettes.html

wt_col = vir20[0]
het_col = vir20[10]
hom_col = vir20[19]
miss_col = COLORS[19]

sns.palplot([wt_col, het_col, hom_col, miss_col])

In [ ]:
# Update the coloraxis to make a plot with the new colors

new_colors = [(0 / 4, wt_col), (1 / 4, wt_col),
              (1 / 4, het_col), (2 / 4, het_col),
              (2 / 4, hom_col), (3 / 4, hom_col),
              (3 / 4, miss_col), (4 / 4, miss_col)]

fig.layout.coloraxis.colorscale = new_colors
fig

In [ ]:
# Show 3 heatmaps side-by-side to help with cluster interpretation and help identify potential ADO clones qualitatively
# First get the order of ids and barcodes
dna = sample.dna
sortby = 'NGT'
bars = dna.clustered_barcodes(orderby=sortby)
ids = dna.clustered_ids(orderby=sortby)
# Draw the three heatmaps as before
fig_ngt = dna.heatmap(attribute='NGT', bars_order=bars, features=ids)
fig_dp = dna.heatmap(attribute='AF_MISSING', bars_order=bars, features=ids)
fig_gq = dna.heatmap(attribute='GQ', bars_order=bars, features=ids)
# Now draw them in one plot
import missionbio.mosaic.utils as mutils
import matplotlib.pyplot as plt
# Use matplotlib for the plot
# Change figsize to make it larger or smaller
# In the jupyter notebook, you might have to double click the image to zoom in
fig, axs = plt.subplots(1, 3, figsize=(30, 10))
mutils.static_fig(fig_ngt, ax=axs[0])
mutils.static_fig(fig_dp, ax=axs[1])
mutils.static_fig(fig_gq, ax=axs[2])
# Remove the borders
axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
plt.tight_layout()
# Show the plot
plt.show()

In [ ]:
#plot heatmap using NGT
fig = sample.dna.heatmap(attribute='NGT_FILTERED')
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
#rename clusters based on heatmap representation. rename clusters identically  that are to be merged into one and discarded (e.g. FP)
#sample.dna.rename_labels(
#  {
#    '1': 'TET2/WT1_916',
#    '2': 'FP',
#    '3': 'FP',
#    '4': 'TET2',
#    '5': 'TET2/WT1_914',
#    '6': 'TET2/WT1_916/GATA2',
#    '7': 'TET2/WT1_914/WT1_944',
#    '8': 'TET2/WT1_914/FLT3_251',
#    'missing': 'FP', #false positive
#    'small': 'FP',
#  }
#)

In [ ]:
#redraw heatmap  w/ new labels
fig = sample.dna.heatmap(attribute='NGT')
#fig.layout.coloraxis.colorscale = new_colors
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
#remove barcodes (clones) from data based on renamed labels
#fp_barcodes = sample.dna.barcodes("FP")
#sample.dna = sample.dna.drop(fp_barcodes)
#set(sample.dna.get_labels()) 

In [ ]:
#redraw heatmap
fig = sample.dna.heatmap(attribute='NGT')
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
#redraw heatmap w/ new colorcode and based on samples
fig = sample.dna.heatmap(attribute='NGT',splitby="sample_name")
fig.layout.coloraxis.colorscale = new_colors
go.Figure(fig)
#fig.write_image("heatmap.snps.variants.png")

In [ ]:
#evaluate new total number of cells after above-filtering
sample.dna.shape

In [ ]:
#compared to original cell number
sample.dna.shape

#### Conclusion

    1. Basic filtering of barcodes ids demonstrated
    2. Basic DNA filtering functionality showcased

### CNV Analysis

Preliminary heatmap of CNV shows that there could be two clusters

<b>Topics covered</b>
1. Down sampling options and their effects

In [ ]:
help(sample.cnv.scatterplot)

#### PCA options

    Here the UMAP options are kept constant
    The only parameter in PCA is the number of components

    Here we see how to determine this value, and the effect
    when we deviate from this value

In [ ]:
#Filtering Amplicons
# This returns the reads in each cell and amplicon
reads = sample.cnv.get_attribute('read_counts', constraint='row+col')
reads

In [ ]:
# Only amplicons found in more than half the cells are analyzed 
# The other amplicons are dropped.
working_amplicons = (reads.median() > 0).values
sample.cnv = sample.cnv[:, working_amplicons]

In [ ]:
# We are now left with fewer amplicons
sample.cnv.shape

In [ ]:
# only valid barcodes from DNA and store at sample.cnv
sample.cnv = sample.cnv[sample.dna.barcodes(),:]

In [ ]:
sample.cnv.shape

In [ ]:
sample.cnv.normalize_reads()
#fig = sample.cnv.heatmap(attribute='normalized_counts')
#go.Figure(fig)
#fig.write_image("heatmap.amplicons.cnv.png")

In [ ]:
help(sample.cnv.normalize_reads)

In [ ]:
# Assuming WT cells are diploid for all the amplicons,
# we can compute the ploidy of the other cell lines as follows

#sample.cnv.compute_ploidy(diploid_cells=sample.dna.barcodes('TET2'))

In [ ]:
# Assign the DNA labels to the CNV object
# We want to ensure the labels are the same

sample.cnv.set_labels(sample.dna.get_labels())
sample.cnv.set_palette(sample.dna.get_palette())

In [ ]:
# Heatmap with the features ordered chromosomal location

#fig = sample.cnv.heatmap('ploidy', features='positions')
#fig

In [ ]:
# Scale the figure width and plot as a static image.
# Double click on the plot to zoom-in and improve the resolution

import missionbio.mosaic.utils as mutils

fig = sample.cnv.heatmap('ploidy', features='genes')
fig.layout.width = 1600
mutils.static_fig(fig, figsize=(20, 20))

In [ ]:
# Heatmap for a subset of the chromosomes

fig = sample.cnv.heatmap('ploidy', features=['7', '17', '11'])
fig

In [ ]:
# Heatmap with the features grouped by the genes
# The first time this runs, it fetches the gene names from ensembl
# The annotation can also be fetched using using sample.cnv.get_gene_names()
# The plots can also be smoothed using a moving average with the convolve parameter

fig = sample.cnv.heatmap('ploidy', features='genes', convolve=2)
fig

In [ ]:
# Change the color scale to "magma" - other suitable options might be "viridis", "plasma", "blues", "blues_r"...
fig.layout.coloraxis.colorscale = 'magma'

# Update the separating lines to be black
for shape in fig.layout.shapes:
    shape.line.color = '#000000'

# Set the minimum value to 0 and maximum value of ploidy to 2
fig.layout.coloraxis.cmax = 2
fig.layout.coloraxis.cmin = 0

fig

In [ ]:
# Heatmap for a subset of the genes
# Change the color scale to "magma" - other suitable options might be "viridis", "plasma", "blues", "blues_r"...
fig = sample.cnv.heatmap('ploidy', features=['EZH2', 'BRAF', 'KMT2A','SAMD9','SAMD9L'])
fig.layout.coloraxis.colorscale = 'magma'

# Update the separating lines to be black
for shape in fig.layout.shapes:
    shape.line.color = '#000000'

# Set the minimum value to 0 and maximum value of ploidy to 2
fig.layout.coloraxis.cmax = 2
fig.layout.coloraxis.cmin = 0
fig

In [ ]:
# WT clone cells normalized to ploidy = 2 by default
sample.cnv.plot_ploidy('TET2')

In [ ]:
#evaluate ploidy for other genotype-based defined clones by updating the code
sample.cnv.plot_ploidy('TET2/WT1_914')

In [ ]:
#evaluate ploidy for other genotype-based defined clones by updating the code
sample.cnv.plot_ploidy('TET2/WT1_916/GATA2')

In [ ]:
sample.cnv.col_attrs

In [ ]:
#if genotype-based clustering does not yield useful CNV data, cluster cnv data (reads) unbiased using the 
# PCA/UMAP clustering
sample.cnv.normalize_reads()
sample.cnv.run_pca(components=300, attribute='normalized_counts',show_plot=True)

In [ ]:
#if genotype-based clustering does not yield useful CNV data, cluster cnv data (reads) unbiased using the 
# PCA/UMAP clustering (Note, all cell barcodes are now used, including cells that were previously filtered out
#due to incomplete genotype data)

sample.cnv.run_pca(components=50, attribute='normalized_counts',show_plot=False)

In [ ]:
#run UMAP on top of the newly created PC dataframe. 
#See https://jlmelville.github.io/uwot/abparams.html for appropriate spread/min_dist values
sample.cnv.run_umap(attribute='pca')#, min_dist=0.2, spread=1.5, random_state=40)
#visualize data and cluster it using different methods
sample.cnv.cluster(attribute='umap', method='graph-community')#, eps=0.8)

In [ ]:
fig = sample.cnv.scatterplot(attribute='umap',colorby='label')
go.Figure(fig)


In [ ]:
sample.cnv.row_attrs

In [ ]:
fig = sample.cnv.scatterplot(attribute='umap',colorby='sample_name')
go.Figure(fig)


In [ ]:
sample.cnv.heatmap(attribute='normalized_counts')#,splitby='sample_name')

In [ ]:
help(sample.cnv.heatmap)

#### Conclusion

    Given all other variables are kept constant

    1. Too many PCA components may result in merging of clusters
    2. Too few PCA component may result in splitting of clusters
    3. The appropriate number of components can be determined using the elbow plot

### Protein Analysis

<b>Topics covered</b>
1. Basic workflow
2. Custom clustering eg. selection on biaxial plot
3. Custom methods by adding layers

#### Basic workflow

In [ ]:
help(sample.protein.normalize_reads)

In [ ]:
# Clustering similar to "cluster method #2" of DNA clustering using genotypes (sample.dna)
sample.protein.normalize_reads('CLR') 
# three different methods avlb: CLR = center-log-transformed here

In [ ]:
sample.protein.run_pca(attribute='normalized_counts', components=45,show_plot=True) # components = 10 appropriate for 45-plex BL panel
#select fewer number of components when analyzing custom AOC panels with 2-20.

In [ ]:
sample.protein.run_pca(attribute='normalized_counts', components=12, show_plot=True) # components = 10 appropriate for 45-plex BL panel
#select fewer number of components when analyzing custom AOC panels with 2-20.

In [ ]:
#UMAPs rely on an initial randomization. 
#This leads to different projections every time. To address this, pass random_state to the run_umap method
sample.protein.run_umap(attribute='pca',spread=2,min_dist=0.4,random_state=42)
#See https://jlmelville.github.io/uwot/abparams.html for appropriate spread/min_dist values

In [ ]:
sample.protein.cluster(attribute='umap', method='graph-community',k=100) 
# the higher the k value, the smaller the number of clones --> adjust if necessary

In [ ]:
help(sample.protein.heatmap)

In [ ]:
fig = sample.protein.heatmap(attribute='normalized_counts')
go.Figure(fig)

In [ ]:
#relabel clone names acccording to biology, combine clones by assigned identical names
sample.protein.rename_labels(
    {
        '14': '',
        '13': '',
        '6': '',
        '9': '',
        '1': '',
        '3': '',
        '10': '',
        '12': '',
        '8': '',
        '4': '',
        '5': '',
        '7': '',
        '11': '',
        '2': '',
    }
)

In [ ]:
#run heatmap again w/ new labels
fig = sample.protein.heatmap(attribute='normalized_counts')
go.Figure(fig)

In [ ]:
#UMAP
fig = sample.protein.scatterplot(attribute='umap',colorby='label')
go.Figure(fig)

In [ ]:
#drop clones (barcodes) that need to be filtered. instead of overwriting sample.protein variable, we define new 
#variable sample.protein. Note if defining sample.protein to update the code in the subsequent steps.
fp_barcodes2 = sample.protein.barcodes("FP")
sample.protein = sample.protein.drop(fp_barcodes2)
set(sample.protein.get_labels())

In [ ]:
#use only if no barcodes are removed by the above-listed command
#sample.protein = sample.protein

In [ ]:
#re-run heatmap with new filtered cells
fig = sample.protein.heatmap(attribute='normalized_counts')
go.Figure(fig)

In [ ]:
sample.protein.ids() #use sample.protein.ids() alternatively

In [ ]:
#remove AOCs that don't display any signal in any cells and overwrite sample.protein variable
#sample.protein = sample.protein.drop('CD90')

In [ ]:
sample.protein.ridgeplot(attribute='normalized_counts',
                         features=sample.protein.ids())

In [ ]:
# UMAP with the expression for each of the selected protein overlayed
# In case of error, make sure that ids have been selected on the heatmap and shown in sample.protein.selected_ids

fig = sample.protein.scatterplot(attribute='umap',
                           colorby='normalized_counts',
                           features=['Vista','CD8','CD3','CD15'])
go.Figure(fig)

#### Custom clustering

    When `labels=False` for any scatterplot
    the lasso tool can be used to cluster cells
    based on the selection made

In [ ]:
# Selction on biaxial scatterplot
# The same can be done for UMAP when labels=False is passed

fig = sample.protein.feature_scatter(layer='normalized_counts', ids=['CD3', 'Vista'],colorby='label')

go.Figure(fig)

In [ ]:
help(sample.protein.scatterplot)

#### Custom methods by adding layers

    If someone is interested in trying their methods,
    they can simply modify the appropriate layers, attributes
    and metadata to plugin their step in this workflow

In [ ]:
# Custom normalization by changing the `normalized_counts` layer

import numpy as np

log_reads = np.log10(10 + sample.protein.layers['read_counts'])
norm = np.divide(log_reads, log_reads.mean(axis=1).reshape(-1, 1))


sample.protein.add_layer('normalized_counts', norm)

    Other examples include:
    
    custom labels -> 'label' row_attr
    custom palette -> 'palette' metadata   

#### Conclusion

    1. Protein analysis workflow similar to CNV
    2. Different clustering methods can result in
       different types of clusters being identified
    3. It is possible to have custom clustering for
       any scatterplot by using the lasso tool
    4. Custom analysis is possible by modifying appropriate
       layers, attributes and metadata

### Statistics

    The significane of differential expression
    based on a t-test can be looked at using
    the `feature_signature` method

In [ ]:
med, std, pval, tstat = sample.protein.feature_signature(layer='normalized_counts')

In [ ]:
pval

In [ ]:
pval = pval + 10 ** -50 + pval
pvals = -np.log10(pval) * (tstat > 0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 10))
fig = sns.heatmap(pvals.T, vmax=50, vmin=40)
#go.Figure(fig)

### Combined visualizations

    Visualization for multiple assays at once

In [ ]:
help(sample.clone_vs_analyte)

In [ ]:
fig = sample.clone_vs_analyte('cnv')

In [ ]:
# In this case, only a set of barcodes are required

# Choosing only two cell lines
cells = sample.dna.barcodes(['TET2',
 'TET2/WT1_914',
 'TET2/WT1_914/FLT3_251',
 'TET2/WT1_914/WT1_944',
 'TET2/WT1_916',
 'TET2/WT1_916/GATA2'])
filtered_sample = sample[cells]

In [ ]:
# This is showing all the variants, minus the false positives (ADO and small clones)

fig = filtered_sample.clone_vs_analyte(analyte='protein')

In [ ]:
filtered_sample.clone_vs_analyte('cnv')

In [ ]:
### Filtering protein and cnv to improve the visualization

filtered_sample.protein = filtered_sample.protein[:, ['CD117', 'CD11A', 'CD123', 'CD16', 'CD45RA','CD34','CD8','CD33' ]]
#sample.cnv = sample.cnv[:, 58:85]
filtered_sample.clone_vs_analyte('protein')

In [ ]:
# The ids can also be reset to the entire set

filtered_sample = sample[cells]

In [ ]:
fig = filtered_sample.heatmap(clusterby='protein', sortby='dna', drop='cnv', flatten=False)
# sample.heatmap(clusterby='protein', sortby='dna', drop='cnv', flatten=False)
# sample.heatmap(clusterby='dna', sortby='protein', flatten=False)
go.Figure(fig)

Often the number of amplicons in CNV might take over the sample level heatmap making the plot uninterpretable. Moreover there might be certain non-differentiating variants and protein in the panel. These can be dropped before making the final heatmap.

In [ ]:
help(filtered_sample.heatmap)

In [ ]:
# Filter the CNV with amplicons only from the relevant genes

import numpy as np

genes = filtered_sample.cnv.col_attrs['gene_name'].copy()
relevant_ids = np.isin(genes, ['EZH2','BRAF','SAMD9','SAMD9L'])

filtered_sample.cnv = filtered_sample.cnv[:, relevant_ids]

In [ ]:
fig = filtered_sample.heatmap(clusterby='dna', sortby='cnv',drop='protein', flatten=False)

# Update the width of the plot [See the section on CNV heatmaps]
fig.layout.width = 3000

# Change the CNV colorscale [See the section on CNV heatmaps]
fig.data[2].zmax = 2
fig.data[2].zmin = 0
fig.data[2].colorscale = 'magma'

# Updating the ticktexts to show the gene names instead
fig.layout.xaxis3.ticktext = filtered_sample.cnv.col_attrs['gene_name'].copy()

# Show as a static plot
mutils.static_fig(fig, figsize=(20, 20))

In [ ]:
#export data
import os

## set directory that does NOT exist
folder_to_save = '/Users/robert_durruthy/Downloads/new-folder'

os.mkdir(folder_to_save)
for assay in [sample.dna, sample.cnv, sample.protein]:
    if assay is not None:
        os.mkdir(f'{folder_to_save}/{assay.name}')
        os.mkdir(f'{folder_to_save}/{assay.name}/layers')
        os.mkdir(f'{folder_to_save}/{assay.name}/rows')

        for layer in assay.layers.keys():
            df = assay.get_attribute(layer, constraint='row+col')
            cols = list(df.columns.values)
            df.loc[:, 'label'] = assay.get_labels()
            df = df.loc[:, ['label'] + cols]
            df.to_csv(f'{folder_to_save}/{assay.name}/layers/{layer}.csv')
        
        for row in assay.row_attrs.keys():
            df = assay.get_attribute(row, constraint='row')
            cols = list(df.columns.values)
            df.loc[:, 'label'] = assay.get_labels()
            df = df.loc[:, ['label'] + cols]
            df.to_csv(f'{folder_to_save}/{assay.name}/rows/{row}.csv')

In [ ]:
#fishplot DNA Requires multiple timepoints (merged h5 files on Tapestri Pipeline V2)
fig = filtered_sample.dna.fishplot()
go.Figure(fig)

In [ ]:
#fishplot protein Requires multiple timepoints (merged h5 files on Tapestri Pipeline V2)
fig = filtered_sample.protein.fishplot()
go.Figure(fig)

    Conclusion:
    
    Data from h5 files can be effeciently manipulated,
    visualized, and inferred using this multiomics
    exploratory tools (mextools)